In [1]:
import oclock
from threading import Event
from clivo import CommandLineInterface, ControlledProperty, ControlledEvent

# Info on what object properties will be controlled

In [2]:
interval = ControlledProperty(attribute='timer.interval',
                              readable='Δt (s)',
                              commands=('dt',))

averaging = ControlledProperty(attribute='averaging',
                               readable='Averaging',
                               commands=('avg',))
            
value = ControlledProperty(attribute='value',
                           readable='Value',
                           commands=('val',))

# Generate objects to be controlled

In [3]:
class TestObjectA:

    def __init__(self):
        self.timer = oclock.Timer(interval=1, warnings=True)
        self._avg = 10  
        self.controlled_properties = interval, averaging
        
    @property
    def averaging(self):
        return self._avg

    @averaging.setter
    def averaging(self, val):
        self._avg = val
        self.timer.reset()  # for immediate effect

    def on_stop(self):
        self.timer.stop()
        
class TestObjectB:

    def __init__(self):     
        self.averaging = 10
        self.value = 1    
        self.controlled_properties = value, averaging

In [4]:
obj1 = TestObjectA()
obj2 = TestObjectA()
obj3 = TestObjectB()

objects = {'A1': obj1, 'A2': obj2, 'objB': obj3}

# Indicate what events can be triggered from CLI

In [5]:
graph_event = ControlledEvent(event=Event(),
                              readable='graph',
                              commands=('g', 'graph'))
                    
events = graph_event,

(Note: a stop event is generated internally but can also be passed)

# Create and start command line interface

In [6]:
cli = CommandLineInterface(objects=objects, events=events)

Stop event not passed. Creating one internally.


In [7]:
cli.run()

OBJECTS ========================================================================
--- A1 [<__main__.TestObjectA object at 0x108653400>]
        Δt (s)
        Averaging
--- A2 [<__main__.TestObjectA object at 0x108653b80>]
        Δt (s)
        Averaging
--- objB [<__main__.TestObjectB object at 0x108653d00>]
        Value
        Averaging
COMMANDS =======================================================================
--- Properties
        avg -- Averaging [averaging]
        val -- Value [value]
        dt -- Δt (s) [timer.interval]
--- Events
        g, graph -- graph
--- Exit
        q, Q, quit
EXAMPLE ========================================================================
avg-A2 xx -- change Averaging to xx for A2 only
avg xx -- change Averaging to xx for all relevant objects
Δt (s)
A1-----1
A2-----1
objB---N/A
New Δt (s) for A1: 3
Unknown Command. 
Δt (s)
A1-----3
A2-----1
objB---N/A
CLI stopped


In [8]:
obj1.timer.interval

3